In [2]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

Xtrain = np.array(pd.read_csv('X.train.csv'))
Xtest = np.array(pd.read_csv('X.test.csv'))

# from sklearn import preprocessing
# Xtrain = preprocessing.scale(Xtrain) 

ytrain = np.array(pd.read_csv('y.train.csv')['label'])

imp = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
imp.fit(Xtrain)

Xtrain = imp.transform(Xtrain)
Xtest = imp.transform(Xtest)

Xtrain = (Xtrain - Xtrain.mean()) / (Xtrain.max() - Xtrain.min())
Xtest = (Xtest - Xtest.mean()) / (Xtest.max() - Xtest.min())
print Xtrain.shape, Xtest.shape

from sklearn.cross_validation import LeaveOneOut


from sklearn.grid_search import GridSearchCV

svr = svm.SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
gamma=0.0, kernel='sigmoid', max_iter=-1, probability=True)
# knc = KNeighborsClassifier(n_neighbors=500, p=1)
# rfc = RandomForestClassifier(n_estimators=200, criterion='entropy', min_samples_split=6, max_depth=1, min_samples_leaf=6)
# gbc = GradientBoostingClassifier(n_estimators=200, min_samples_split=5, max_depth=1, min_samples_leaf=6)
# abc = AdaBoostClassifier(rfc, n_estimators=200, algorithm="SAMME")
# parameters = {'min_samples_split':range(1, 10), 'min_samples_leaf':range(5, 10), 'max_depth':range(6, 15)}
# parameters = {'p': [(i + 2)*0.5 for i in xrange(40)]}
parameters = {'coef0': range(-2, 2)}

(4099L, 1330L) (1366L, 1330L)


In [3]:
gs = GridSearchCV(svr, parameters, n_jobs = 1,scoring = 'log_loss', cv = 5, verbose = 5)

In [4]:
gs.fit(Xtrain[:, :331], ytrain)
ypred = gs.predict_proba(Xtest[:, :331])[:, 1]

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] coef0=-2 ........................................................
[CV] .............................. coef0=-2, score=-0.288538 -   6.3s
[CV] coef0=-2 ........................................................
[CV] .............................. coef0=-2, score=-0.288783 -   6.0s
[CV] coef0=-2 ........................................................
[CV] .............................. coef0=-2, score=-0.288783 -   6.1s
[CV] coef0=-2 ........................................................
[CV] .............................. coef0=-2, score=-0.286108 -   6.3s
[CV] coef0=-2 ........................................................
[CV] .............................. coef0=-2, score=-0.286108 -   6.0s
[CV] coef0=-1 ........................................................
[CV] .............................. coef0=-1, score=-0.288538 -   5.4s
[CV] coef0=-1 ........................................................
[CV] ............

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    6.3s
[Parallel(n_jobs=1)]: Done  18 jobs       | elapsed:  1.8min



[CV] coef0=1 .........................................................
[CV] ............................... coef0=1, score=-0.286108 -   5.8s
[CV] coef0=1 .........................................................
[CV] ............................... coef0=1, score=-0.286108 -   5.6s


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  2.0min finished


In [4]:
gs.best_estimator_
# gs.best_estimator_+
# Out[15]:
# RandomForestClassifier(bootstrap=True, compute_importances=None,
#             criterion='entropy', max_depth=13, max_features='auto',
#             max_leaf_nodes=None, min_density=None, min_samples_leaf=6,
#             min_samples_split=5, n_estimators=200, n_jobs=1,
#             oob_score=False, random_state=None, verbose=0)
# GradientBoostingClassifier(min_samples_split=6, max_depth=1, min_samples_leaf=7)
# SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.4,
#   kernel='rbf', max_iter=-1, probability=True, random_state=None,
#   shrinking=True, tol=0.001, verbose=False)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.4,
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [34]:
dfpred = pd.DataFrame()
dfpred['Id'] = np.arange(len(ypred))
dfpred['Prediction'] = ypred

dfpred.to_csv('y.predicted.csv', index = False)